In [2]:
from pathlib import Path
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [3]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [4]:
query = """
select distinct(tipo_beneficiario) from cae"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,NUEVO BENEFICIARIO
1,EGRESO
2,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
3,DESERCION
4,BENEFICIARIO RENOVANTE
5,GRATUIDAD


In [ ]:
query_new = """
WITH summary_table as (
    SELECT
        id_2023 AS id,
        genero_registrado AS genero, 
        region_grupo_familiar AS region,
        quintil_ingreso_grupo_familiar AS quintil,
        tipo_ies AS tipo_establecimiento,
        region_sede_ies AS region_estudio,
        tipo_titulo AS tipo_carrera,
        dependencia_colegio AS tipo_colegio,
        nombre_carrera as carrera,
        nombre_ies as establecimiento,

        MIN(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN año_licitacion END)
            AS año_inicio_cae,

        MIN(CASE WHEN tipo_beneficiario = 'DESERCION' THEN año_operacion END)
            AS año_desercion,

        COUNT(CASE WHEN tipo_beneficiario IN ('BENEFICIARIO RENOVANTE','NUEVO BENEFICIARIO')
                THEN tipo_beneficiario END)
            AS años_financiados,

        MIN(CASE WHEN tipo_beneficiario = 'EGRESO' THEN año_operacion END)
            AS año_egreso,

        MIN(CASE WHEN tipo_beneficiario = 'GRATUIDAD' THEN año_operacion END)
            AS año_gratuidad,

        CASE 
            WHEN SUM(CASE WHEN cambio_ies_carrera = 'SI' THEN 1 ELSE 0 END) > 0
                THEN 'SI'
            ELSE 'NO'
        END AS cambio_ies_carrera,

    FROM cae
    GROUP BY
        id, genero, region, quintil, tipo_establecimiento, region_estudio,
        tipo_carrera, tipo_colegio, carrera, establecimiento),
    
    total_años_financiados as (
        SELECT id, SUM(años_financiados) as total_años_financiados
        FROM summary_table
        GROUP BY id
        )
    SELECT st.*, taf.total_años_financiados
    FROM summary_table st
    LEFT JOIN total_años_financiados taf
        ON st.id = taf.id
        """

summary_table = executes_query_duckdb(query_new, cae)

In [ ]:
summary_table

,id,genero,region,quintil,tipo_establecimiento,region_estudio,tipo_carrera,tipo_colegio,carrera,establecimiento,año_inicio_cae,año_desercion,años_financiados,año_egreso,año_gratuidad,cambio_ies_carrera,total_años_financiados
0,254.299,M,REGION DE LA ARAUCANIA,quintil 4,U. PRIVADA - NO CRUCH,9,Profesional con Licenciatura,Particular pagado,ODONTOLOGIA,UNIVERSIDAD MAYOR,2009,<NA>,8,2017,<NA>,NO,8.0
1,432.827,M,REGION DE LA ARAUCANIA,quintil 1 y 2,IP,9,Profesional,Municipal,CONSTRUCCION CIVIL,INSTITUTO PROFESIONAL AIEP,<NA>,<NA>,2,2017,<NA>,NO,4.0
2,433.429,M,REGION DEL BIO-BIO,quintil 1 y 2,IP,8,Profesional,Municipal,"INGENIERIA EN MAQUINARIA, VEHICULOS AUTOMOTRIC...",INSTITUTO PROFESIONAL INACAP,2014,2017,2,<NA>,<NA>,NO,2.0
3,224.445,F,REGION DEL BIO-BIO,quintil 1 y 2,IP,8,Técnico,Particular subvencionado,TECNICO EN ENFERMERIA,INSTITUTO PROFESIONAL DR. VIRGINIO GÓMEZ G.,2009,<NA>,3,2013,<NA>,NO,3.0
4,194.650,F,REGION DEL LIB. B. O'HIGGINS,quintil 1 y 2,IP,6,Técnico,Particular subvencionado,TECNICO DE NIVEL SUPERIOR EN ODONTOLOGIA,INSTITUTO PROFESIONAL AIEP,2011,<NA>,2,2013,<NA>,NO,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681570,1.197.351,M,REGION DE ÑUBLE,quintil 3,CRUCH,8,Profesional con Licenciatura,Particular Subvencionado,INGENIERIA CIVIL MECANICA,UNIVERSIDAD DEL BÍO-BÍO,2024,<NA>,1,<NA>,<NA>,NO,1.0
1681571,1.201.651,M,REGION METROPOLITANA,quintil 4,IP,13,Técnico,Municipal DAEM,ANALISTA PROGRAMADOR COMPUTACIONAL,INSTITUTO PROFESIONAL DUOC UC,2024,<NA>,1,<NA>,<NA>,NO,1.0
1681572,1.155.656,M,REGION METROPOLITANA,quintil 5,CFT,13,Técnico,Particular Subvencionado,TECNICO EN ADMINISTRACION DE RECURSOS HUMANOS,CENTRO DE FORMACIÓN TÉCNICA SANTO TOMÁS,<NA>,<NA>,1,<NA>,<NA>,NO,2.0
1681573,1.227.806,F,REGION METROPOLITANA,quintil 4,U. PRIVADA - NO CRUCH,13,Bachiller,Particular Pagado,INGENIERIA COMERCIAL PLAN COMUN,UNIVERSIDAD ADOLFO IBÁÑEZ,2024,<NA>,1,<NA>,<NA>,NO,1.0
